In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import re
import yaml
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sequence_gan_max_ent_tab import init
from postprocessing import write_sample
import os
import pickle

In [3]:
%load_ext tensorboard
%tensorboard --logdir runs

In [93]:
#clean empty folders
for git_dir in Path("runs").iterdir():
    empty_folder = True
    for run_dir in git_dir.iterdir():
        f =  list(run_dir.rglob("*checkpoint*")) + list(run_dir.rglob("*events.out.tfevents.*"))\
                +list(run_dir.rglob("*out*")) + list(run_dir.rglob("*err*"))
        if f:
            empty_folder = False
        else:
            try:
                #shutil.rmtree(run_dir)
                #print("succesfully removed: {}".format(run_dir))
            except:
                print("error occured while removing: {}".format(run_dir))
            
            
    if empty_folder :
        print("is empty")
        #shutil.rmtree(git_dir)

error occured while removing: runs/hpc-job-log/6576590.out
error occured while removing: runs/hpc-job-log/6576564.out
error occured while removing: runs/hpc-job-log/6576577.out
error occured while removing: runs/hpc-job-log/6576590.err
error occured while removing: runs/hpc-job-log/6576564.err
error occured while removing: runs/hpc-job-log/6576577.err
is empty
error occured while removing: runs/ff9350c52ad954b6547edbde62a0b58733c5a632-dirty/20200429-104956
error occured while removing: runs/cc5ecb4ecf512321e3f37ef5ad4944dded6a9d35-dirty/20200504-103710
error occured while removing: runs/cc5ecb4ecf512321e3f37ef5ad4944dded6a9d35-dirty/20200504-105905
error occured while removing: runs/cc5ecb4ecf512321e3f37ef5ad4944dded6a9d35-dirty/20200504-103305
error occured while removing: runs/cc5ecb4ecf512321e3f37ef5ad4944dded6a9d35-dirty/20200504-103833
is empty


In [42]:
def config_file_filter(config_filter,path_filter=lambda x:True):
    config_files = list(Path("runs").rglob("*config.yaml"))
    file_list = []
    for config_file in config_files:
        if path_filter(config_file):
            with open(config_file) as stream:
                config = yaml.load(stream, Loader=yaml.Loader)
                if config_filter(config):
                    file_list.append(config_file)
    return file_list
def files2regex(config_files):
    tags = ""
    for config_file in config_files:
        if tags:
            tags += "|"
        tags += str(re.escape(str(config_file.relative_to('runs').parent.joinpath('log'))))
    return tags

In [49]:
def config_filter(config):
    return config['is_music_data']
def path_filter(path):
    return str(path).find("4cf70fa3b65cac12229f02484914379fec9b2f55-dirty/20200512") != -1 
print(
    files2regex(
        config_file_filter(config_filter,path_filter)
    )
)

4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-172846/log|4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-171258/log|4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-171437/log|4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-172937/log|4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-171902/log|4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-171924/log|4cf70fa3b65cac12229f02484914379fec9b2f55\-dirty/20200512\-171708/log


In [63]:
#with open('runs/4cf70fa3b65cac12229f02484914379fec9b2f55-dirty/20200512-172937/config.yaml') as stream: #0.5 ent, good lr normal orange
#with open('runs/4cf70fa3b65cac12229f02484914379fec9b2f55-dirty/20200512-171924/config.yaml') as stream: # green
with open('runs/4cf70fa3b65cac12229f02484914379fec9b2f55-dirty/20200512-172846/config.yaml') as stream: #grey
    config = yaml.load(stream, Loader=yaml.Loader)
    print(config['generator_lr'])

0.0001


# Load model

In [46]:
run = 'runs/49b1df8786702568d88efc00e30cec550e9ec40a-dirty/20200503-184420'
tf.compat.v1.reset_default_graph()
gan_trainer, config = init(run + '/config.yaml')
saver = tf.compat.v1.train.Saver()
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=tf_config)
saver.restore(sess, run + '/model/advtrain')

KeyError: 'rewards_reduced_variance'

# Generator

In [28]:
out = 'runs/91549835c94af6d3c2984e66864617a948d41b77-dirty/20200512-142933'

In [140]:
samples = gan_trainer.generator.generate(sess)

In [160]:
write_sample(np.array(samples[10]), out + '/midi/test.mid')

### Train Generator for testing 

In [28]:
gan_trainer.advtrain_gen(sess,1,config['rollout_num'],config['ent_temp'])

174.12189

### BLEU Score

In [36]:
gan_trainer.calculate_bleu(sess, gan_trainer.generator, gan_trainer.eval_data_loader)

0.37656923751196036

### LL score

In [29]:
gan_trainer.cross_p_q_2(sess)

3.8624465

### G-loss

In [30]:
gan_trainer.rollout.normalize

True

In [25]:
gan_trainer.rollout.update_rate = 0
gan_trainer.rewards_reduced_variance = False

In [12]:
samples = gan_trainer.generator.generate(sess)

In [69]:
rewards = gan_trainer.rollout.get_reward(sess, samples, config['rollout_num'], gan_trainer.discriminator, ent_temp=config['ent_temp'])

In [32]:
rewards2 = gan_trainer.rollout.get_reward_2(sess, samples, config['rollout_num'], gan_trainer.discriminator, ent_temp=config['ent_temp'])

In [64]:
rewards

array([[0.39245725, 0.3924575 , 0.39245752, ..., 0.392457  , 0.39245725,
        0.39245737],
       [0.27170154, 0.27170166, 0.2717017 , ..., 0.27170137, 0.27170163,
        0.27170157],
       [0.3320792 , 0.33207935, 0.3320793 , ..., 0.33207914, 0.33207935,
        0.33207932],
       ...,
       [0.39245415, 0.3924542 , 0.39245433, ..., 0.3924543 , 0.39245418,
        0.39245412],
       [0.2113207 , 0.21132076, 0.21132089, ..., 0.21132082, 0.2113207 ,
        0.21132056],
       [0.33207622, 0.33207628, 0.3320763 , ..., 0.33207634, 0.33207607,
        0.33207598]], dtype=float32)

In [66]:
np.min(rewards)

-4.437767

In [103]:
gen = gan_trainer.generator
feed = {gan_trainer.generator.x: gan_trainer.generator.generate(sess),
             gan_trainer.generator.rewards: rewards}
grads_op = tf.gradients(gan_trainer.generator.g_loss, gan_trainer.generator.g_params) 
grads = sess.run(grads_op,
         feed)

#sess.run(tf.norm(tf.clip_by_global_norm(tf.gradients(gan_trainer.generator.g_loss, gan_trainer.generator.g_params)
#                               ,5.0)[0][10]),feed)
#sess.run(g_opt.apply_gradients(list(zip(
#    tf.clip_by_global_norm(tf.gradients(gen.g_loss, gen.g_params)[0], gen.grad_clip)
#    , gen.g_params
#))),feed)

In [105]:
clipped_grad, glob_norm = sess.run(tf.clip_by_global_norm(grads_op,5.0),feed)


In [106]:
glob_norm

2336.6118

In [ ]:
glob_norm

In [102]:
grads[0].shape

AttributeError: 'IndexedSlicesValue' object has no attribute 'shape'

In [99]:
(np.square(grads[0])).shape

(3,)

In [96]:
ss = 0
for grad in grads:
    ss += np.sum(np.square(grad))
np.sqrt(ss)

ValueError: operands could not be broadcast together with shapes (3232,32) (3232,) 

In [72]:
rewards.dtype

dtype('float32')

In [64]:
feed = {gan_trainer.generator.x: samples, gan_trainer.generator.rewards: rewards}
sess.run(gan_trainer.generator.g_loss, feed_dict=feed)

345.50574

 on same sample 
using normalize
rollout 32:
345.50574
199.06557
rollout 1: 107.4561

In [52]:
np.mean(rewards,axis=0)

array([ 1.5032589e-03, -2.7861893e-03, -2.6963770e-02, -8.5195899e-03,
       -8.8212788e-03,  6.9578290e-03, -2.1509603e-03,  1.3257410e-02,
        1.4136843e-02, -3.4468770e-03, -2.1138713e-03, -3.9737746e-03,
        2.4257302e-03,  2.5183123e-02,  3.5549775e-02,  1.5003920e-02,
        1.8955112e-02,  4.0849075e-03,  2.5410801e-02, -1.1112392e-03,
        1.5194207e-02, -8.3371997e-04, -5.7371065e-02, -6.2846392e-04,
        7.4505806e-09, -1.5395284e-03,  5.5132806e-04, -7.1117692e-03,
       -4.3213367e-03,  3.1592436e-03,  1.3136119e-02,  1.0881277e-02,
        8.4388256e-04,  3.1807654e-02,  4.9584210e-03, -3.3264607e-03,
        2.6755035e-03, -7.9460517e-03, -6.1948672e-03,  8.9965761e-04,
        6.2675327e-03, -3.6852956e-03,  4.1498691e-03,  1.5313178e-03,
       -1.0355890e-02,  4.9601123e-03, -4.5971349e-03, -4.0403232e-03,
       -1.1131145e-02,  1.2039304e-02,  2.3280457e-03, -1.0720149e-02,
        5.3905845e-03,  1.1411607e-03,  3.2376722e-03,  1.7032862e-02,
      

# Discriminator

In [40]:
discriminator = gan_trainer.discriminator
gan_trainer.dis_data_loader.load_train_data(config['positive_file'],run + '/save/generator_sample.txt')
x_batch, y_batch = gan_trainer.dis_data_loader.next_batch()
feed = {
    discriminator.input_x: x_batch,
    discriminator.input_y: y_batch,
    discriminator.dropout_keep_prob: 1.0
}
nll = sess.run(tf.nn.softmax_cross_entropy_with_logits(
    logits=discriminator.scores, labels=discriminator.input_y
), feed)
print("nll: {}".format(np.mean(nll)))
probs = np.exp(-nll)
pos_index = list(map(lambda x: (x == np.array([0,1])).all(), y_batch))
neg_index = list(map(lambda x: (x == np.array([1,0])).all(), y_batch))
print("Probability of True positive: {}".format(probs[pos_index]))
print("Probability of True negative: {}".format(probs[neg_index]))
TP_index = probs[pos_index] > 0.5
TP = len((probs[pos_index])[TP_index])
FP_index = probs[neg_index] < 0.5
FP =  len((probs[neg_index])[FP_index])
print("Precision: {}".format(TP/(TP+FP)))
print("Recall: {}".format(TP/float(len(probs[pos_index]))))

nll: 9.007888793945312
Probability of True positive: [1.         0.99999976 1.         1.         0.9999988  0.99999833
 1.         0.9999999  1.         0.99999726 0.9999919  0.99999833
 1.         0.9999999  0.9999999  1.        ]
Probability of True negative: [2.94108759e-07 1.82739029e-06 1.62054661e-07 2.02947533e-07
 1.69242576e-08 1.38412375e-08 1.69878955e-09 2.19157179e-10
 4.96180519e-09 2.43527751e-07 3.39326767e-09 1.34033922e-08
 8.49034052e-08 1.65241133e-07 5.52989876e-10 9.90389715e-12]
Precision: 0.5
Recall: 1.0


In [271]:
FP_index = probs[neg_index] < 0.5
FP =  len((probs[neg_index])[FP_index])
FP

1

In [250]:
x_batch[pos_index]

array([[  50,  135,  944, ...,    2,   38, 1839],
       [  46,   45,   56, ...,  497,   47,    5],
       [ 112,  504,  114, ...,  590,  173,  122],
       ...,
       [ 488,  944,  215, ...,   47, 2619,    4],
       [   4,   50,   36, ...,   38,  114,  114],
       [  35,   47,   99, ...,  111,  116,  104]])

In [34]:
feed = {
    discriminator.input_x: x_batch,
    discriminator.dropout_keep_prob: 1.0
}
predictions = sess.run(discriminator.predictions, feed)

In [35]:
predictions[pos_index]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
predictions[neg_index]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [168]:
np.log(1)

0.0

# BLEU

In [108]:
!ls 'runs/91549835c94af6d3c2984e66864617a948d41b77-dirty/20200506-133756/midi'

ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
Ep_1_test_0.mid


In [81]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu,  SmoothingFunction
import numpy as np
smoother = SmoothingFunction()
bleu_avg = 0

references = [["there","is","a","cat","on","the","mat"],["the" ,"cat","is","on","the","mat"],]
hypotheses = [["the", "the", "cat"],["there", "is", "a", "cat"]]

hyp_i = 1
bleu =  sentence_bleu(references, hypotheses[hyp_i], weights=[1.0]) # smoothing_function=smoother.method4)
bleu_corp = corpus_bleu(references, hypotheses, weights=[1.0]) 
#bleu = 0.
# calculate bleu for each predicted seq
# compare each predicted seq with the entire references
# this is slow, use multiprocess
#def calc_sentence_bleu(hypothesis):
#    return sentence_bleu(references, hypothesis, smoothing_function=smoother.method4)

In [82]:
hypotheses[hyp_i]

['there', 'is', 'a', 'cat']

In [83]:
r = 6

In [52]:
bleu_corp

0.14285714285714285

In [53]:
BP = np.exp(1-6/4)
BP

0.6065306597126334

In [49]:
bleu/BP

1.0

In [84]:
np.exp(1- (r+r)/(3+4))

0.48954165955695317

In [86]:
BP_corp = np.exp(1- (r+r)/(3+4))
BP_corp

0.48954165955695317

In [87]:
bleu_corp/BP_corp

0.2918181528951631

In [28]:
((2+1)+(1+1+1+1))/(3+3)

1.1666666666666667

In [16]:
sum([len(ref) for ref in references])

13

In [17]:
(1.0,1.0)

(1.0, 1.0)

In [5]:
!ls dataset

ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
Notthingham.zip  chords   dataset2   octaves	tokens	valid
Nottingham	 dataset  generated  prep_data	train	valid copy


# Dataset 

In [8]:
train = pickle.load(open('dataset/train','rb'))

In [10]:
len(train)

1400

In [16]:
len(train[10])

100

In [23]:
ls

ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
Inspect variables.ipynb                    hpc_non_graphic.sh*
NONAME_6572117.out                         hpc_non_graphic.sh~*
README.md                                  midi/
SeqGAN_Music.yaml~                         postprocessing.py
Tabular multimodal train GAN simple.ipynb  requirements.txt
Tabular multimodal train GAN.ipynb         rollout.py
Tabular multimodal.ipynb                   rollout_max_ent.py
Tensorboard.ipynb                          runs/
__pycache__/                               save/
configs/                                   seqgan-6572151.out
core.8876                                  seqgan_6572151.err
dataloader.py                              sequence_gan.py
dataset/                                   sequence_gan.py.bak
discriminator.py                           sequence_g

In [14]:
!ls -l dataset/Nottingham/train| egrep -c '^-'

ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
694


In [22]:
!ls -l dataset/Nottingham/valid| egrep -c '^-'

ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
173


In [13]:
!ls dataset/Nottingham/train

ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/usr/lib64/libGL.so.1.2.0' from LD_PRELOAD cannot be preloaded: ignored.
ashover_simple_chords_1.mid    reels_simple_chords_120.mid
ashover_simple_chords_12.mid   reels_simple_chords_121.mid
ashover_simple_chords_13.mid   reels_simple_chords_122.mid
ashover_simple_chords_14.mid   reels_simple_chords_123.mid
ashover_simple_chords_15.mid   reels_simple_chords_126.mid
ashover_simple_chords_18.mid   reels_simple_chords_127.mid
ashover_simple_chords_19.mid   reels_simple_chords_128.mid
ashover_simple_chords_2.mid    reels_simple_chords_129.mid
ashover_simple_chords_20.mid   reels_simple_chords_13.mid
ashover_simple_chords_21.mid   reels_simple_chords_132.mid
ashover_simple_chords_24.mid   reels_simple_chords_133.mid
ashover_simple_chords_25.mid   reels_simple_chords_134.mid
ashover_simple_chords_26.mid   reels_simple_chords_135.mid
ashover_simple_chords_27.mid   reels_simple_